In [1]:
## from https://github.com/yunjey/pytorch-tutorial

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch.utils.data as utils

In [2]:
# input_size = 12       # The image size = 28 x 28 = 784
# hidden_size = 500      # The number of nodes at the hidden layer
# num_classes = 2       # The number of output classes. In this case, from 0 to 9
# num_epochs = 5         # The number of times entire dataset is trained
# batch_size = 100       # The size of input data took for one iteration
# learning_rate = 0.000001  # The speed of convergence

In [3]:
# Import Dataset
qcd_raw = pd.read_csv('../HiggsReconstruction/EventPlotting/qcd_outputDataForLearning.csv')
hh_raw = pd.read_csv('../HiggsReconstruction/EventPlotting/dihiggs_outputDataForLearning.csv')

qcd_raw.head()
print(len(qcd_raw), "rows of qcd data")
hh_raw.head()
print(len(hh_raw), "rows of dihiggs data")

# Make higgs and qcd sets from raw data
# hh_all = hh_raw[['h1_mass', 'h2_mass', 'deltaR(h1 jets)', 'deltaR(h2 jets)', 'jet1_pz', 'jet2_pz', 'jet3_pz', 'jet4_pz', 'jet1_energy', 'jet2_energy', 'jet3_energy', 'jet4_energy']]
# qcd = qcd_raw[['h1_mass', 'h2_mass', 'deltaR(h1 jets)', 'deltaR(h2 jets)', 'jet1_pz', 'jet2_pz', 'jet3_pz', 'jet4_pz', 'jet1_energy', 'jet2_energy', 'jet3_energy', 'jet4_energy']]
hh_all = hh_raw[['deltaR(h1 jets)', 'deltaR(h2 jets)']]
qcd = qcd_raw[['deltaR(h1 jets)', 'deltaR(h2 jets)']]
n_factors = np.shape(hh_all)[1]
print(n_factors)

1703 rows of qcd data
4605 rows of dihiggs data
2


In [5]:
hh_all = np.array(hh_all)
qcd = np.array(qcd)

# add labels to di-higgs
hh_all=hh_all[:,:n_factors]
# # print(hh[0:3])
hh_labels= np.zeros((len(hh_all),1))
hh_labels = hh_labels+1
#a = hh[:len(hh)]
# print(a.shape)
hh_all[:,0] = np.random.rand(np.shape(hh_all)[0])
hh_all = np.append(hh_all, hh_labels, axis=1)
hh_all = np.append(hh_all, 1-hh_labels, axis=1)## hh qcd labels 


# print(hh.shape)
# print(hh[0:3])

# add labels to qcd
qcd=qcd[:,:n_factors]
# print(hh[0:3])
qcd_labels= np.zeros((len(qcd),1))
#a = hh[:len(hh)]
# print(a.shape)
# qcd hh labels 
qcd[:, 0] = -1 * np.random.rand(np.shape(qcd)[0])
qcd = np.append(qcd, qcd_labels, axis=1) 
qcd = np.append(qcd, 1-qcd_labels, axis=1)# qcd qcd labels


# use this for dummy variables
hh_all[:,0] = np.random.rand(np.shape(hh_all)[0])
hh_all[:,1] = np.random.rand(np.shape(hh_all)[0])
# qcd[:, 0] = -1 * np.random.rand(np.shape(qcd)[0])
# qcd[:, 1] = -1 * np.random.rand(np.shape(qcd)[0])
# qcd[:, 0] = np.random.rand(np.shape(qcd)[0])
# qcd[:, 1] = np.random.rand(np.shape(qcd)[0])
qcd[:, 0] = hh_all[:len(qcd),0]
qcd[:, 1] = hh_all[:len(qcd),1]


# select a quarter of hh events so that the set is half and half
# we shuffle the list first to take a random 1/4. this means we have a different dataset every time
# np.random.seed(0)
# np.random.shuffle(hh_all) 
hh = hh_all[0:len(qcd)]
# print(hh[:4])
# print(qcd[:4])

all_data = np.append(hh,qcd, axis=0) 
all_data[:n_factors,:]

np.random.seed(0)
for i in range (4): # shuffle 4 times
    np.random.shuffle(all_data) 
print(all_data[:4])
all_labels = all_data[:,n_factors:]
# for testing model resilience
# for i in range(2):
#     np.random.shuffle(all_labels)
all_data = all_data[:,:n_factors]
# print(all_data[:4])
print(all_labels[:4])
# print(test_data)
# print(len(all_data))
# print(all_labels)

input_size = n_factors       # The image size = 28 x 28 = 784
hidden_size = 500      # The number of nodes at the hidden layer
num_classes = all_labels.shape[1]       # The number of output classes. In this case, from 0 to 9
num_epochs = 5         # The number of times entire dataset is trained
batch_size = 100       # The size of input data took for one iteration
learning_rate = 0.001  # The speed of convergence

[[0.70037731 0.9287528  1.         0.        ]
 [0.35012013 0.49221131 0.         1.        ]
 [0.05149961 0.24521989 1.         0.        ]
 [0.81105927 0.400845   1.         0.        ]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [6]:
# scale the data by dividing it by the max value of each
for i in range(np.shape(all_data)[1]):
    all_data[:,i] = np.true_divide(all_data[:,i], np.max(all_data[:,i]))
print(all_data[:4])

[[0.70042873 0.92880387]
 [0.35014584 0.49223837]
 [0.05150339 0.24523337]
 [0.81111881 0.40086704]]


In [7]:
train_data, test_data, train_labels, test_labels = train_test_split(all_data, all_labels, train_size=0.3, test_size=0.5, random_state=42)

train_data = torch.stack([torch.Tensor(i) for i in train_data])
train_labels = torch.stack([torch.Tensor(i) for i in train_labels])
test_data = torch.stack([torch.Tensor(i) for i in test_data])
test_labels = torch.stack([torch.Tensor(i) for i in test_labels])



train_dataset = utils.TensorDataset(train_data, train_labels)

test_dataset = utils.TensorDataset(test_data, test_labels)

print(train_dataset)

In [8]:

train_loader = utils.DataLoader(train_dataset)

test_loader = utils.DataLoader(test_dataset)
print(train_loader)

In [9]:
print(enumerate(train_loader))
for i, e in enumerate(train_loader):
    print(i, e)
    if(i>4):
        break

0 [tensor([[0.5523, 0.1416]]), tensor([[1., 0.]])]
1 [tensor([[0.9918, 0.0296]]), tensor([[0., 1.]])]
2 [tensor([[0.1546, 0.9431]]), tensor([[0., 1.]])]
3 [tensor([[0.3340, 0.8230]]), tensor([[0., 1.]])]
4 [tensor([[0.3084, 0.7789]]), tensor([[1., 0.]])]
5 [tensor([[0.9753, 0.7578]]), tensor([[1., 0.]])]


In [10]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [11]:
net = Net(input_size, hidden_size, num_classes)


In [12]:
# net.cuda()    # You can comment out this line to disable GPU

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [14]:
for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        data = Variable(data)       # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(data)                             # Forward pass: compute the output class given a image
        labels = labels.long()
        loss = criterion(outputs, torch.max(labels, 1)[1])                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
#             print("SMH")
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data.item()))
print(len(train_dataset))

Epoch [1/5], Step [100/10], Loss: 0.6699
Epoch [1/5], Step [200/10], Loss: 0.6800
Epoch [1/5], Step [300/10], Loss: 0.9490
Epoch [1/5], Step [400/10], Loss: 1.3598
Epoch [1/5], Step [500/10], Loss: 0.6646
Epoch [1/5], Step [600/10], Loss: 0.6152
Epoch [1/5], Step [700/10], Loss: 0.5246
Epoch [1/5], Step [800/10], Loss: 0.6436
Epoch [1/5], Step [900/10], Loss: 1.3878
Epoch [1/5], Step [1000/10], Loss: 1.5333
Epoch [2/5], Step [100/10], Loss: 0.8062
Epoch [2/5], Step [200/10], Loss: 0.7949
Epoch [2/5], Step [300/10], Loss: 0.9589
Epoch [2/5], Step [400/10], Loss: 1.2454
Epoch [2/5], Step [500/10], Loss: 0.6573
Epoch [2/5], Step [600/10], Loss: 0.6173
Epoch [2/5], Step [700/10], Loss: 0.4721
Epoch [2/5], Step [800/10], Loss: 0.6579
Epoch [2/5], Step [900/10], Loss: 1.2035
Epoch [2/5], Step [1000/10], Loss: 1.4511
Epoch [3/5], Step [100/10], Loss: 0.7677
Epoch [3/5], Step [200/10], Loss: 0.8147
Epoch [3/5], Step [300/10], Loss: 0.9768
Epoch [3/5], Step [400/10], Loss: 1.1707
Epoch [3/5], S

In [15]:
correct = 0
total = 0
for data, labels in test_loader:
    data = Variable(data)
    outputs = net(data)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    labels = labels.long()
    correct += (predicted == torch.max(labels, 1)[1]).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))
print(correct)
print(total)

Accuracy of the network on the 10K test images: 49 %
tensor(846)
1703
